## Assignment 9

In [1]:
import numpy as np
import math
from textwrap import fill

## Theory

### Fixed Size Coding


In [2]:
Message = 'BCCABBDDAECCBBAEDDCC'
l = len(Message)

frequency = {}
for i in Message:
    if i not in frequency.keys():
        frequency[i] = 1
    else:
        frequency[i] += 1
        
bits = int((math.log(len(frequency)) / math.log(2)) + 1)
    
print('\t   TABLE')
for i,j in frequency.items():
    ans = (bin(ord(i)- ord('A')))[2:].zfill(bits)
    print(i, '\t',j, '\t', j/l, '\t', ans)
    
print('\n\nNumber of letters in the Message = {}\n'.format(l))

print('Length of each bit if send without Encoding = {}'.format(8))
print('finale size of message to be sent = {} * 8 = {} bits\n'.format(l, l*8))

print('ENCODED MESSAGE SIZE : \nLength of each bit if send with Encoding = {}'.format(bits))
print('Space needed for the Encoded Message = {} * {} = {}'.format(l, bits, l*bits))
print('\nTABLE SIZE : \nSize of Character = {} * 8 = {} bits'.format(len(frequency), len(frequency)*8))
print('Size for Code = {} * {} = {} bits'.format(len(frequency), bits, len(frequency)*bits))
print('Extra space to send the sub table = {} + {} = {} bits'.format(len(frequency)*8, len(frequency)*bits, len(frequency)*(bits+8)))
print('Finale size of Message + Table to be sent = {} * {} = {} bits'.format(len(frequency)*(bits+8), l*bits, len(frequency)*(bits+8) + l*bits))

	   TABLE
B 	 5 	 0.25 	 001
C 	 6 	 0.3 	 010
A 	 3 	 0.15 	 000
D 	 4 	 0.2 	 011
E 	 2 	 0.1 	 100


Number of letters in the Message = 20

Length of each bit if send without Encoding = 8
finale size of message to be sent = 20 * 8 = 160 bits

ENCODED MESSAGE SIZE : 
Length of each bit if send with Encoding = 3
Space needed for the Encoded Message = 20 * 3 = 60

TABLE SIZE : 
Size of Character = 5 * 8 = 40 bits
Size for Code = 5 * 3 = 15 bits
Extra space to send the sub table = 40 + 15 = 55 bits
Finale size of Message + Table to be sent = 55 * 60 = 115 bits


### Variable Size Coding - (HUFFMAN Coding)

In [3]:
class HuffmanTree(object):
    def __init__(self, left=None, right=None):
        self.left = left
        self.right = right

    def children(self):
        return (self.left, self.right)

    def nodes(self):
        return (self.left, self.right)

    def __str__(self):
        return "%s_%s" % (self.left, self.right)

## Tansverse the NodeTress in every possible way to get codings
def huffmanCodeTree(node, left=True, binString=""):
    if type(node) is str:
        return {node: binString}
    (l, r) = node.children()
    tree = dict()
    tree.update(huffmanCodeTree(l, True, binString + "0"))
    tree.update(huffmanCodeTree(r, False, binString + "1"))
    return tree

def huffmanCoding(string):
    frequencyList = {}
    for char in string:
        if char in frequencyList: #count frequency of char in the string
            frequencyList[char] += 1
        else:
            frequencyList[char] = 1

    # Sort the frequency table based on occurrence this will also convert the
    # dict to a list of tuples
    frequencyList = sorted(frequencyList.items(), key=lambda x: x[1], reverse=True)

    nodes = frequencyList

    while len(nodes) > 1:
        keyLeft, freqLeft = nodes[-1]
        keyRight, freqRight = nodes[-2]#
        nodes = nodes[:-2]#removes first 2 nodes from PQ
        parentNode = HuffmanTree(keyLeft, keyRight)
        nodes.append((parentNode, freqLeft + freqRight))
        nodes = sorted(nodes, key=lambda x: x[1], reverse=True)# Re-sort the list
    
    huffmanCode = huffmanCodeTree(nodes[0][0])#start running the tree by the root

    #PRINT
    print("\n Character | Frequency  | Huffman code ")
    print("-----------------------------------------")
    for char, frequency in frequencyList:
        print(" %-9r | %10d | %12s" % (char, frequency, huffmanCode[char]))

    count = 0
    str = ""
    #print("\nThe encoded string is:")
    for char in string:
        #print(huffmanCode[char], end='')
        str += huffmanCode[char]
        count += len(huffmanCode[char])
    print("\nThe encoded string is:")
    print(fill(str, width=75))
    print("\n\nThe total length of coding is:")
    print(count)
    
    return frequencyList, huffmanCode

def getString(filename):  # Get graph from file
    file = open(filename, "r", encoding='utf-8')
    lines = file.readlines()
    string = ' '.join(lines)
    return string

## Q1

In [4]:
probs = [0.25, 0.25, 0.125, 0.125, 0.125, 0.0625, 0.0625]
stringFile = getString('Message.txt')
print('\nInput : Message.txt')
frequency, huff = huffmanCoding(stringFile)


Input : Message.txt

 Character | Frequency  | Huffman code 
-----------------------------------------
 'A'       |          4 |           01
 'B'       |          4 |           00
 'C'       |          2 |          101
 'D'       |          2 |          100
 'E'       |          2 |          111
 'F'       |          1 |         1101
 '\n'      |          1 |         1100

The encoded string is:
010101010000000010110110010011111111011100


The total length of coding is:
42


In [5]:
# Average Code Length
avg_code_len = 0
for i in range(7):
    avg_code_len += probs[i] * len(huff[frequency[i][0]])
    
# Entropy
entropy = 0
for i in range(7):
    entropy += probs[i]*math.log(probs[i], 2.0)
    
# Efficiency
effeciency = (-entropy / avg_code_len) * 100
    
avg_code_len, -entropy, effeciency

(2.625, 2.625, 100.0)

## Q2

### LZW  Encoding

In [6]:
def search_dic(x, dic):
    for i, j in dic.items():
        if x == j:
            return i
    

message = 'ababbabcababba'
encoded = []
l = len(message)
dic = {1:'a', 2:'b', 3:'c'}
c = 3
i, j = 0, 0
while i < l and j < l:
    if message[i:j+1] in dic.values():
        j += 1
    else:
        c += 1
        dic[c] = message[i:j+1]
        
        encoded.append(search_dic(message[i:j], dic))
        i += j-i

encoded.append(search_dic(message[i:j], dic))

In [7]:
encoded

[1, 2, 4, 5, 2, 3, 4, 6, 1]

### LZW Decoding

In [74]:
dic = {1:'a', 2:'b', 3:'c'}
l = len(encoded)

message = []

entry = {}
partial = {}
c = 4


for i in range(l):
    if encoded[i] in dic.keys():
        message.append(dic[encoded[i]])
        
        if i != 0:
            entry[search_dic(list(partial.values())[-1], partial)] = list(partial.values())[-1] + dic[encoded[i]]
            
        partial[c] = dic[encoded[i]]
        
    elif encoded[i] in entry.keys():
        message.append(entry[encoded[i]])
        
        entry[search_dic(list(partial.values())[-1], partial)] = list(partial.values())[-1] + message[-1][0]
            
        partial[c] = entry[encoded[i]]
        
    c += 1

In [75]:
message

['a', 'b', 'ab', 'ba', 'b', 'c', 'ab', 'abb', 'a']